# **Installing libraries**
---

## **Importing libraries**

In [1]:
import pandas as pd
import json
import google.generativeai as genai

C:\Users\Manjit\AppData\Local\Temp\ipykernel_6580\2043909488.py:3: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


## **API Configuration**

### **Retrieveing API KEY**

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not set in environment.")

### **Checking for availabel models**

In [3]:
# Setting API KEY
genai.configure(api_key=GEMINI_API_KEY)

# Getting available models list
models = list(genai.list_models())

# Printing first 5 models
for m in models[:5]:
    print(m.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation


### **Selecting model**

In [4]:
model = genai.GenerativeModel("gemini-2.5-flash")

# **Loading and Processing Dataset**
---

## **Loading dataset**

In [5]:
df = pd.read_csv("data/titanic_data.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## **Schema Extraction Function**

In [6]:
def extract_schema(df):
    schema = []

    for col in df.columns:
        schema.append({
            "column_name": col,
            "dtype": str(df[col].dtype),
            "non_null_count": int(df[col].notnull().sum()),
            "unique_values": int(df[col].nunique())
        })

    return {
        "num_rows": len(df),
        "num_columns": len(df.columns),
        "columns": schema,
        "sample_rows": df.head(2).to_dict(orient="records")
    }

### **Extracting schema and inspecting**

In [7]:
dataset_info = extract_schema(df)

print(json.dumps(dataset_info, indent=1))

{
 "num_rows": 418,
 "num_columns": 12,
 "columns": [
  {
   "column_name": "PassengerId",
   "dtype": "int64",
   "non_null_count": 418,
   "unique_values": 418
  },
  {
   "column_name": "Survived",
   "dtype": "int64",
   "non_null_count": 418,
   "unique_values": 2
  },
  {
   "column_name": "Pclass",
   "dtype": "int64",
   "non_null_count": 418,
   "unique_values": 3
  },
  {
   "column_name": "Name",
   "dtype": "str",
   "non_null_count": 418,
   "unique_values": 418
  },
  {
   "column_name": "Sex",
   "dtype": "str",
   "non_null_count": 418,
   "unique_values": 2
  },
  {
   "column_name": "Age",
   "dtype": "float64",
   "non_null_count": 332,
   "unique_values": 79
  },
  {
   "column_name": "SibSp",
   "dtype": "int64",
   "non_null_count": 418,
   "unique_values": 7
  },
  {
   "column_name": "Parch",
   "dtype": "int64",
   "non_null_count": 418,
   "unique_values": 8
  },
  {
   "column_name": "Ticket",
   "dtype": "str",
   "non_null_count": 418,
   "unique_values": 3

# **Prompt Construction**
---

## **Defining prompt**

In [8]:
def build_prompt(dataset_info, question):
    return f"""
You are a senior data analyst.

You are given:
1. Dataset metadata
2. A user question

Your task:
- Generate SAFE pandas code using ONLY dataframe `df`
- Do NOT import anything
- Do NOT assign variables
- Return a JSON object with keys:
    - analysis_type
    - pandas_code
    - reasoning

Rules:
- pandas_code must be a SINGLE expression
- No loops
- No comprehensions
- No file access
- No __ usage
- Only pandas operations on df

Dataset Metadata:
{json.dumps(dataset_info, indent=2)}

User Question:
{question}

Return ONLY valid JSON.
"""

# **Calling Gemini and Parsing Output**
---

## **LLM Call Function**

In [9]:
def query_gemini(prompt):
    response = model.generate_content(prompt)
    return response.text

## **Structured Parsing**

In [10]:
import re
import json

def extract_json_block(text):
    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1:
        return text[start:end+1]
    return text
    
def parse_llm_output(raw_output):
    try:
        json_str = extract_json_block(raw_output)
        return json.loads(json_str)
    except Exception as e:
        print("JSON Parsing Failed:", e)
        print("Raw Output:\n", raw_output)
        return None

# **Testing**
---

In [11]:
question = "What as the total number of passengers onboard?"

prompt = build_prompt(dataset_info, question)
raw_output = query_gemini(prompt)

print("Raw Output")
print("-"*50)
print(raw_output)

print("Parsed LLM Output")
print("-"*50)
parsed = parse_llm_output(raw_output)
parsed

Raw Output
--------------------------------------------------
```json
{
  "analysis_type": "summary",
  "pandas_code": "len(df)",
  "reasoning": "The user is asking for the total number of passengers onboard, which directly corresponds to the total number of rows in the DataFrame. The `len()` function applied to a DataFrame returns its number of rows."
}
```
Parsed LLM Output
--------------------------------------------------


{'analysis_type': 'summary',
 'pandas_code': 'len(df)',
 'reasoning': 'The user is asking for the total number of passengers onboard, which directly corresponds to the total number of rows in the DataFrame. The `len()` function applied to a DataFrame returns its number of rows.'}

# **Building safe execution layer**
---

## **Basic Static Safety Filter**

In [12]:
import ast

FORBIDDEN_KEYWORDS = [
    "import",
    "__",
    "exec",
    "eval",
    "open",
    "write",
    "read",
    "os",
    "sys",
    "subprocess",
    "pickle",
    "to_csv",
    "to_excel"
]

def basic_safety_check(code):
    for word in FORBIDDEN_KEYWORDS:
        if word in code:
            raise ValueError(f"Unsafe keyword detected: {word}")

## **AST Validation**

In [13]:
def ast_validate(code):
    try:
        tree = ast.parse(code, mode="eval")  # only expressions allowed
    except:
        raise ValueError("Code is not a valid single expression.")

    for node in ast.walk(tree):

        if isinstance(node, (ast.Import, ast.ImportFrom)):
            raise ValueError("Imports not allowed.")

        if isinstance(node, ast.Call):
            if isinstance(node.func, ast.Name):
                if node.func.id not in ["len", "sum", "min", "max"]:
                    # Allow pandas method calls via df only
                    pass

        if isinstance(node, ast.Name):
            if node.id not in ["df", "len", "sum", "min", "max"]:
                raise ValueError(f"Unauthorized variable: {node.id}")

    return True

## **Automatic Repair Loop**

In [14]:
def execute_with_repair(parsed_output, df, dataset_info, question, max_retries=2):
    
    current_output = parsed_output
    
    for attempt in range(max_retries + 1):
        
        code = current_output["pandas_code"]
        
        try:
            result = safe_execute(code, df)
            return result, current_output
        
        except Exception as e:
            
            print(f"Attempt {attempt+1} failed.")
            print("Error:", str(e))
            
            if attempt == max_retries:
                raise RuntimeError("Max retries exceeded.")
            
            # Build repair prompt
            repair_prompt = f"""
The following pandas expression failed:

Expression:
{code}

Error:
{str(e)}

Dataset Metadata:
{json.dumps(dataset_info, indent=2)}

User Question:
{question}

Return corrected JSON with:
- analysis_type
- pandas_code
- reasoning

Return ONLY valid JSON.
"""
            
            raw = query_gemini(repair_prompt)
            repaired = parse_llm_output(raw)
            
            if repaired is None:
                raise RuntimeError("Repair output not parseable.")
            
            current_output = repaired

## **Adding Result Normalizer**

In [15]:
import numpy as np
import pandas as pd

def normalize_result(result):
    
    # Convert numpy scalars to native Python
    if isinstance(result, (np.integer, np.int64)):
        return int(result)
    
    if isinstance(result, (np.floating, np.float64)):
        return float(result)
    
    # Convert pandas Series
    if isinstance(result, pd.Series):
        return result.to_dict()
    
    # Convert pandas DataFrame
    if isinstance(result, pd.DataFrame):
        return result.to_dict(orient="records")
    
    return result

## **Safe Execution**

In [16]:
def safe_execute(code, df):
    basic_safety_check(code)
    ast_validate(code)

    allowed_builtins = {
        "len": len,
        "sum": sum,
        "min": min,
        "max": max
    }

    safe_globals = {"__builtins__": allowed_builtins}
    safe_locals = {"df": df}

    try:
        result = eval(code, safe_globals, safe_locals)
        return normalize_result(result)
    except Exception as e:
        raise RuntimeError(f"Execution error: {e}")

## **Building Exaplaination Prompt**

In [17]:
def build_explanation_prompt(question, pandas_code, result):
    
    return f"""
You are a data analyst.

The user asked:
{question}

The system executed this pandas expression:
{pandas_code}

The computed result is:
{result}

Explain the result clearly in simple English.

Be concise.
Do not mention internal system details.
"""

## **Explaination function**

In [18]:
def generate_explanation(question, pandas_code, result):
    
    prompt = build_explanation_prompt(question, pandas_code, result)
    
    response = model.generate_content(prompt)
    
    return response.text.strip()

In [19]:
question = "Out of the people who survived, how many were males and how many were females?"

prompt = build_prompt(dataset_info, question)
raw_output = query_gemini(prompt)
parsed = parse_llm_output(raw_output)
parsed

result, final_output = execute_with_repair(parsed, df, dataset_info, question)

explanation = generate_explanation(
    question,
    final_output["pandas_code"],
    result
)

print("\nAnswer:")
print(explanation)


Answer:
Among the people who survived, 152 were females. No males were found.


---